In [16]:
from janome.tokenizer import Tokenizer

t = Tokenizer()
t

In [17]:
text = '東京都でおいしいビールを飲もう。'
# トークンをジェネレータからリストへ変換
tokens = list(t.tokenize(text))

# リストの長さを取得
print(len(tokens))

# トークンを出力してみる
for token in tokens:
    print(token)

9
東京	名詞,固有名詞,地域,一般,*,*,東京,トウキョウ,トーキョー
都	名詞,接尾,地域,*,*,*,都,ト,ト
で	助詞,格助詞,一般,*,*,*,で,デ,デ
おいしい	形容詞,自立,*,*,形容詞・イ段,基本形,おいしい,オイシイ,オイシイ
ビール	名詞,一般,*,*,*,*,ビール,ビール,ビール
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
飲も	動詞,自立,*,*,五段・マ行,未然ウ接続,飲む,ノモ,ノモ
う	助動詞,*,*,*,不変化型,基本形,う,ウ,ウ
。	記号,句点,*,*,*,*,。,。,。


In [19]:
from janome.tokenizer import Tokenizer

# Tokenizerのインスタンス化
t = Tokenizer()

text = '東京都でおいしいビールを飲もう。'

# 分かち書きされた単語をジェネレータとして取得
wakati_generator = t.tokenize(text, wakati=True)

# ジェネレータをリストに変換
wakati_words = list(wakati_generator)

# 結果を表示
print(wakati_words)

['東京', '都', 'で', 'おいしい', 'ビール', 'を', '飲も', 'う', '。']


In [23]:
from janome.tokenizer import Tokenizer

t = Tokenizer()
sentences = ['おいしいビールを飲む', 'コーヒーを飲む', 'おいしいクラフトビールを買う']

word_list = []
for sentence in sentences:
    # ジェネレータをリストに変換して追加
    words = list(t.tokenize(sentence, wakati=True))
    word_list.append(words)

print(word_list)


[['おいしい', 'ビール', 'を', '飲む'], ['コーヒー', 'を', '飲む'], ['おいしい', 'クラフト', 'ビール', 'を', '買う']]


In [24]:
unique_words = []
for words in word_list:
    for word in words:
        if word not in unique_words:
            unique_words.append(word)

print(unique_words)

['おいしい', 'ビール', 'を', '飲む', 'コーヒー', 'クラフト', '買う']


In [25]:
bow_list = []
for words in word_list:
    bow = []
    for unique_word in unique_words:
        bow.append(words.count(unique_word))
    bow_list.append(bow)

print(bow_list)

[[1, 1, 1, 1, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0], [1, 1, 1, 0, 0, 1, 1]]


In [26]:
from math import log
num_of_sentences = len(sentences)
idf = []
for i in range(len(unique_words)):
    count = 0
    for words in word_list:
        if unique_words[i] in words:
            count += 1
    idf.append(log((num_of_sentences + 1) / (count + 1)) + 1)

print(idf)

[1.2876820724517808, 1.2876820724517808, 1.0, 1.2876820724517808, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454]


In [27]:
bow = bow_list[1]
num_of_sentences = sum(bow)
tfidf = []
for i in range(len(bow)):
    tfidf.append(bow[i] * idf[i] / num_of_sentences)

print(tfidf)


[0.0, 0.0, 0.3333333333333333, 0.42922735748392693, 0.5643823935199818, 0.0, 0.0]


In [28]:
# 天気予報の各状態を表すデータ
weather_data = {
    '晴れ': {
        'next_weather_types': ['晴れ', '曇り', '雨'],  # 遷移先の天気のリスト
        'weights': [0.6, 0.3, 0.1],  # それぞれの天気に遷移する割合（重み）
    },
    '曇り': {'next_weather_types': ['晴れ', '曇り', '雨'], 'weights': [0.3, 0.5, 0.2]},
    '雨': {'next_weather_types': ['晴れ', '曇り', '雨'], 'weights': [0.2, 0.3, 0.5]},
}

In [29]:
# 状態を表すデータから「晴れ」の次の天気を重みありでランダムに取り出す
import random

next_weather_types = weather_data['晴れ']['next_weather_types']
weights = weather_data['晴れ']['weights']

count = {'晴れ': 0, '曇り': 0, '雨': 0}
for i in range(1000):  # 重みづけを確認するために1000回天気を取り出す
    weather = random.choices(next_weather_types, weights=weights)[0]  # リストで返るので先頭を取り出す
    count[weather] += 1  # 出現回数を数える
print(count)

{'晴れ': 605, '曇り': 299, '雨': 96}


In [33]:
# 天気のマルコフ連鎖を生成する

current_weather = '晴れ'
markov_weather_list = [current_weather]  # 天気のリストを格納する領域
for i in range(10):
    next_weather_types = weather_data[current_weather]['next_weather_types']  # 次の状態（天気）と重みを取得
    weights = weather_data[current_weather]['weights']
    current_weather = random.choices(next_weather_types, weights=weights)[0]  # 次の状態（天気）をランダムに取得
    markov_weather_list.append(current_weather)  # 天気を追加
markov_weather_list

['晴れ', '晴れ', '雨', '晴れ', '晴れ', '晴れ', '晴れ', '晴れ', '雨', '雨', '雨']